# Behavioral model testing

In this notebook, we will apply behavioral model testing to a sentiment analysis model named [SentiStrength](https://mi-linux.wlv.ac.uk/~cm1993/sentistrength/).

We will be calling SentiStrength through the web API of [BehaViz](https://behaviz.com), a sentiment analysis service offered by [PeoplewareAI](https://www.peopleware.ai).

In [1]:
import behaviz_client_improved as bvz
import pandas as pd

In [2]:
bvz.Session.email = bvz.config["auth"]["email"]
bvz.Session.password = bvz.config["auth"]["password"]

## Invariance testing

> *"Changes in the input should not affect model output."*

In [3]:
tokens = ["service", "solution", "product"]
texts = [f"I'm so happy for the excellent {token} you provide!" for token in tokens]

classifier = bvz.BehavizModel.SENTISTRENGTH_EN
task_id = bvz.submit_new_task(texts, classifier)
results = bvz.get_results(task_id)

Token request...
Waiting 0.1 seconds for task completion...


In [9]:
results

[{'sentence': "I'm so happy for the excellent service you provide!",
  'score': {'positive': 5, 'negative': -1},
  'label': 'positive'},
 {'sentence': "I'm so happy for the excellent solution you provide!",
  'score': {'positive': 5, 'negative': -1},
  'label': 'positive'},
 {'sentence': "I'm so happy for the excellent product you provide!",
  'score': {'positive': 5, 'negative': -1},
  'label': 'positive'}]

In [7]:
pd.DataFrame(
    [(result["sentence"], result["score"]["positive"], result["score"]["negative"]) for result in results],
    columns=["Sentence", "Positive", "Negative"],
)

,Sentence,Positive,Negative
0,I'm so happy for the excellent service you pro...,5,-1
1,I'm so happy for the excellent solution you pr...,5,-1
2,I'm so happy for the excellent product you pro...,5,-1


In [12]:
for result in results:
    assert (result["score"]["positive"] + result["score"]["negative"]) > 0

## Directional testing

> *"Changes in the input should affect output."*

In [10]:
tokens = ["love", "hate", "consider"]
texts = [f"I will {token} BCN." for token in tokens]

classifier = bvz.BehavizModel.SENTISTRENGTH_EN
task_id = bvz.submit_new_task(texts, classifier)
results = bvz.get_results(task_id)

Waiting 0.1 seconds for task completion...


In [11]:
pd.DataFrame(
    [(result["sentence"], result["score"]["positive"], result["score"]["negative"]) for result in results],
    columns=["Sentence", "Positive", "Negative"],
)

,Sentence,Positive,Negative
0,I will love BCN.,3,-1
1,I will hate BCN.,1,-4
2,I will consider BCN.,1,-1


In [13]:
# The result is positive
assert results[0]["score"]["positive"] + results[0]["score"]["negative"] > 0

# The result is negative
assert results[1]["score"]["positive"] + results[1]["score"]["negative"] < 0

# The result is neutral
assert results[2]["score"]["positive"] + results[2]["score"]["negative"] == 0

## Minimum functionality testing

> *"Simple combination of inputs and expected outputs."*

In [14]:
tokens = ["Hope", "Despair", "Eggs"]
texts = [f"{token} is all we have left." for token in tokens]

classifier = bvz.BehavizModel.SENTISTRENGTH_EN
task_id = bvz.submit_new_task(texts, classifier)
results = bvz.get_results(task_id)

Waiting 0.1 seconds for task completion...


In [15]:
pd.DataFrame(
    [(result["sentence"], result["score"]["positive"], result["score"]["negative"]) for result in results],
    columns=["Sentence", "Positive", "Negative"],
)

,Sentence,Positive,Negative
0,Hope is all we have left.,3,-1
1,Despair is all we have left.,1,-4
2,Eggs is all we have left.,1,-1


In [16]:
# The result is positive
assert results[0]["score"]["positive"] + results[0]["score"]["negative"] > 0

# The result is negative
assert results[1]["score"]["positive"] + results[1]["score"]["negative"] < 0

# The result is neutral
assert results[2]["score"]["positive"] + results[2]["score"]["negative"] == 0